In [ ]:
import os
import glob
import pickle
###
import pandas as pd
import configparser as cnfp

In [ ]:
config_path = 'config.ini'
config = cnfp.ConfigParser(interpolation=cnfp.ExtendedInterpolation())
config.read(config_path)
#
pick_dir = config.get('Path', 'picks')


pick_name_pattern = config.get('File.Name.Pattern', 'picks')
residual_name_pattern = config.get('File.Name.Pattern', 'residual')

In [ ]:
path_to_residuals = config.get('Path', 'residuals')
os.makedirs(path_to_residuals, exist_ok=True)

In [ ]:
from seisbench.util.annotations import PickList

def read_picks_of_date(path, fname):
    lst_file = glob.glob(f'{path}/{fname}')
    picks = PickList()
    l = len(lst_file)
    for indx, fname in enumerate(lst_file):
        # print(indx, 'of', l)
        with open(fname, 'rb') as fileObj:
            picks += pickle.load(fileObj)
    return picks

def read_picks_of_date_2(path, fname):
    lst_file = glob.glob(f'{path}/{fname}')
    picks = PickList()
    l = len(lst_file)
    for indx, fname in enumerate(lst_file):
        # print(indx, 'of', l)
        with open(fname, 'rb') as fileObj:
            classify = pickle.load(fileObj)
            # print(classify)
            picks += classify.picks
    return picks

In [ ]:
for dl_model in ['EQTransformer', 'PhaseNet']:
    lst_pickle_files = glob.glob(f'{path_to_residuals}/*{dl_model}*')
    lst_df = [pd.read_pickle(fname) for fname in lst_pickle_files]
    df_all = pd.concat(lst_df)
    print('phase in cat: ', df_all.shape[0])
    #
    fname = pick_name_pattern.format(
                net_sta='*',
                date_data='*',
                model_name=dl_model,
                time_name='*'
                )
    picks_dl = read_picks_of_date_2(
            path=pick_dir, fname=fname
        )
    print(dl_model, 'picks:', len(picks_dl))

In [ ]:
lst_pickle_files = glob.glob(f'{path_to_residuals}/*PhaseNet*')
lst_df = [pd.read_pickle(fname) for fname in lst_pickle_files]
df_result = pd.concat(lst_df)
df_result.shape

In [ ]:
df_result